In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, Bidirectional, Dropout, Activation, Multiply

################# Model Creation ####################
def create_model(train, units, dropout=0.2):

    model_LSTM = Sequential()
    model_LSTM.add(LSTM(16,activation='relu',input_shape=(train.shape[1], train.shape[2])))
    model_LSTM.add(Dropout(0.25))
    model_LSTM.add(Dense(1))
    model_LSTM._name='LSTM'

    model_GRU = Sequential()
    model_GRU.add(GRU(16,activation='relu',input_shape=(train.shape[1], train.shape[2])))
    model_GRU.add(Dropout(0.25))
    model_GRU.add(Dense(1))
    model_GRU._name='GRU'

    model_RNN = Sequential()
    model_RNN.add(SimpleRNN(16,activation='relu',input_shape=(train.shape[1], train.shape[2])))
    model_RNN.add(Dropout(0.25))
    model_RNN.add(Dense(1))
    model_RNN._name='RNN'

    model_BiLSTM = Sequential()
    model_BiLSTM.add(Bidirectional(LSTM(8, activation='relu'), input_shape=(train.shape[1], train.shape[2])))
    model_BiLSTM.add(Dropout(0.25))
    model_BiLSTM.add(Flatten())
    model_BiLSTM.add(Dense(1))
    model_BiLSTM._name='BiLSTM'

    model_CNN = Sequential()
    model_CNN.add(Conv1D(filters=16, kernel_size=1,input_shape=(train.shape[1], train.shape[2])))
    model_CNN.add(Flatten())
    model_CNN.add(Dropout(0.25))
    model_CNN.add(Dense(1))
    model_CNN._name='CNN'    
    

    model_proposed1 = Sequential()
    model_proposed1.add(Bidirectional(LSTM(64, activation='relu', return_sequences=True), input_shape=(train.shape[1], train.shape[2])))
    model_proposed1.add(Bidirectional(LSTM(32, activation='relu', return_sequences=True)))
    model_proposed1.add(Bidirectional(LSTM(16, activation='relu', return_sequences=False)))
    model_proposed1.add(Dropout(0.1))
    model_proposed1.add(Dense(1))
    model_proposed1.compile(optimizer='adam', loss='mse')
    model_proposed1._name = 'proposed1'

    model_proposed2 = Sequential()
    model_proposed2.add(Conv1D(filters=16, kernel_size=3, padding='causal', activation='relu', input_shape=(train.shape[1], train.shape[2])))
    model_proposed2.add(Bidirectional(LSTM(64, activation='relu', return_sequences=True)))
    model_proposed2.add(Bidirectional(LSTM(32, activation='relu', return_sequences=True)))
    model_proposed2.add(Bidirectional(LSTM(16, activation='relu', return_sequences=False)))
    model_proposed2.add(Dropout(0.1))
    model_proposed2.add(Dense(1))
    model_proposed2.compile(optimizer='adam', loss='mse')
    model_proposed2._name = 'proposed2'

    model_proposed3 = Sequential()
    model_proposed3.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(train.shape[1], train.shape[2])))
    model_proposed3.add(Bidirectional(LSTM(64, activation='relu', return_sequences=True)))
    model_proposed3.add(Bidirectional(LSTM(32, activation='relu', return_sequences=True)))
    model_proposed3.add(Bidirectional(LSTM(16, activation='relu', return_sequences=False)))
    model_proposed3.add(Dropout(0.1))
    model_proposed3.add(Dense(1))
    model_proposed3.compile(optimizer='adam', loss='mse')
    model_proposed3._name = 'proposed3'

    # Define the WaveNet layer
    def wavenet_layer(inputs, dilation_rate=2, kernel_size=2):
        dilated_conv = Conv1D(filters=32, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(inputs)
        gated_activation = Activation('tanh')(dilated_conv)
        gated_activation = Multiply()([gated_activation, Activation('sigmoid')(dilated_conv)])
        return gated_activation

    # Define the model using the Functional API
    inputs = Input(shape=(train.shape[1], train.shape[2]))
    x = wavenet_layer(inputs)
    x = wavenet_layer(x, dilation_rate=4)
    x = wavenet_layer(x, dilation_rate=8)
    x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
    x = Bidirectional(LSTM(32, activation='relu', return_sequences=True))(x)
    x = Bidirectional(LSTM(16, activation='relu', return_sequences=False))(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)

    model_proposed4 = Model(inputs=inputs, outputs=outputs)
    model_proposed4.compile(optimizer='adam', loss='mse')
    model_proposed4._name = 'proposed4'

    # Define WaveNet model using the Functional API
    def WaveNet(input_shape):
        inputs = Input(shape=input_shape)
        x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
        # Add more layers as per your WaveNet architecture
        outputs = Conv1D(filters=1, kernel_size=1, activation='linear')(x)
        model = Model(inputs=inputs, outputs=outputs)
        return model

    # Example input shape for WaveNet
    wavenet_input_shape = (train.shape[1], train.shape[2])

    # Create WaveNet model
    wavenet_model = WaveNet(wavenet_input_shape)
    wavenet_model.compile(optimizer='adam', loss='mse')

    # Define stacked BiLSTM model
    model_proposed5 = Sequential()
    model_proposed5.add(Bidirectional(LSTM(64, activation='relu', return_sequences=True), input_shape=(train.shape[1], train.shape[2])))
    model_proposed5.add(Bidirectional(LSTM(32, activation='relu', return_sequences=True)))
    model_proposed5.add(Bidirectional(LSTM(16, activation='relu', return_sequences=False)))
    model_proposed5.add(Dropout(0.1))
    model_proposed5.add(Dense(1))
    model_proposed5.compile(optimizer='adam', loss='mse')
    model_proposed5._name = 'proposed5'

    return model_BiLSTM, model_LSTM, model_GRU, model_RNN, model_CNN, model_proposed1, model_proposed2, model_proposed3, model_proposed4, model_proposed5

